# IMDb dataset cleaning

This notebook performs comprehensive cleaning on the IMDb sample dataset and produces two DataFrames: `df_clean` (cleaned) and `df_cast` (exploded by cast).


## Load data


In [2]:
import pandas as pd
import numpy as np

# Load the uploaded sample file
df = pd.read_csv('imdb.csv')
print('Initial shape:', df.shape)
df.head(3)

Initial shape: (10000, 15)


,Poster,Title,Year,Certificate,Duration (min),Genre,Rating,Metascore,Director,Cast,Votes,Description,Review Count,Review Title,Review
0,https://m.media-amazon.com/images/M/MV5BYWRkZj...,The Idea of You,2023.0,R,115.0,"Comedy, Drama, Romance",6.4,67.0,Michael Showalter,"Anne Hathaway, Nicholas Galitzine, Ella Rubin,...","28,744","Solène, a 40-year-old single mom, begins an un...",166,Hypocrisy as an idea,"This film, as well as the reaction to it, is a..."
1,https://m.media-amazon.com/images/M/MV5BZGI4NT...,Kingdom of the Planet of the Apes,2023.0,PG-13,145.0,"Action, Adventure, Sci-Fi",7.3,66.0,Wes Ball,"Owen Teague, Freya Allan, Kevin Durand, Peter ...","22,248","Many years after the reign of Caesar, a young ...",183,A phenomenal start to another trilogy!,"I'm a big fan of all the planet of the apes, a..."
2,https://m.media-amazon.com/images/M/MV5BZjIyOT...,Unfrosted,2023.0,PG-13,97.0,"Biography, Comedy, History",5.5,42.0,Jerry Seinfeld,"Isaac Bae, Jerry Seinfeld, Chris Rickett, Rach...","18,401","In 1963 Michigan, business rivals Kellogg's an...",333,not funny,Pretty much the worst criticism you can lay on...


## Standardize column names

- Lowercase all names
- Replace spaces with underscores
- Remove parentheses tokens like `(min)`
- Rename `duration(min)` to `duration` precisely


In [3]:
# Keep original names for reference
original_cols = df.columns.tolist()

# Normalize: lowercase, spaces->underscores, remove parentheses content like (min)
df.columns = (
    df.columns.str.lower()
             .str.strip()
             .str.replace(' ', '_')
             .str.replace('\n', '_')
             .str.replace('\r', '')
             .str.replace('\t', '_')
             .str.replace('\((min)\)', '', regex=True)
)

# Handle specific duration(min) -> duration case (after normalization it might be 'duration' already or 'duration_min')
rename_map = {}
if 'duration_min' in df.columns:
    rename_map['duration_min'] = 'duration'
if 'duration(min)' in original_cols and 'duration' not in df.columns:
    # In case normalization didn't catch, ensure a final rename
    # (This path is unlikely if normalization worked)
    pass

if 'meta_score' not in df.columns and any(c.lower().strip().replace(' ', '_') == 'meta_score' for c in original_cols):
    # If a weird variant exists, try a broad rename approach
    for c in original_cols:
        if c.lower().strip().replace(' ', '_') == 'meta_score':
            rename_map[c.lower().strip().replace(' ', '_')] = 'meta_score'

df = df.rename(columns=rename_map)

print('Original columns:', original_cols)
print('Standardized columns:', df.columns.tolist())

Original columns: ['Poster', 'Title', 'Year', 'Certificate', 'Duration (min)', 'Genre', 'Rating', 'Metascore', 'Director', 'Cast', 'Votes', 'Description', 'Review Count', 'Review Title', 'Review']
Standardized columns: ['poster', 'title', 'year', 'certificate', 'duration_', 'genre', 'rating', 'metascore', 'director', 'cast', 'votes', 'description', 'review_count', 'review_title', 'review']


## Drop irrelevant columns

Drop review-related text and poster fields: `review_title`, `description`, `review`, `poster`.


In [4]:
drop_cols = ['review_title', 'description', 'review', 'poster']
df = df.drop(columns=[c for c in drop_cols if c in df.columns], errors='ignore')
df.head(3)

,title,year,certificate,duration_,genre,rating,metascore,director,cast,votes,review_count
0,The Idea of You,2023.0,R,115.0,"Comedy, Drama, Romance",6.4,67.0,Michael Showalter,"Anne Hathaway, Nicholas Galitzine, Ella Rubin,...","28,744",166
1,Kingdom of the Planet of the Apes,2023.0,PG-13,145.0,"Action, Adventure, Sci-Fi",7.3,66.0,Wes Ball,"Owen Teague, Freya Allan, Kevin Durand, Peter ...","22,248",183
2,Unfrosted,2023.0,PG-13,97.0,"Biography, Comedy, History",5.5,42.0,Jerry Seinfeld,"Isaac Bae, Jerry Seinfeld, Chris Rickett, Rach...","18,401",333


## Convert types and handle missing values

- Convert `meta_score`, `duration`, and `year` to integers
- Report missing values
- Impute numeric columns with median
- Fill categorical/text columns with `Unknown`


In [5]:
# Identify candidate numeric columns
numeric_candidates = []
for col in ['meta_score', 'duration', 'year']:
    if col in df.columns:
        numeric_candidates.append(col)

# Convert to numeric (coerce errors), then impute median and cast to int
for col in numeric_candidates:
    df[col] = pd.to_numeric(df[col], errors='coerce')
    median_val = df[col].median() if not np.isnan(df[col].median()) else 0
    df[col] = df[col].fillna(median_val).astype(int)

# Report missing after numeric handling
missing_report_before = df.isnull().sum().sort_values(ascending=False)
print('Missing values before categorical fill:\n', missing_report_before)

# Fill object (text) columns with 'Unknown'
for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].fillna('Unknown')

missing_report_after = df.isnull().sum().sort_values(ascending=False)
print('Missing values after fills:\n', missing_report_after)
df.head(3)

Missing values before categorical fill:
 certificate     2630
metascore       2445
rating           404
votes            404
duration_        336
cast              39
genre              7
director           5
review_count       1
title              0
year               0
dtype: int64
Missing values after fills:
 metascore       2445
rating           404
duration_        336
title              0
year               0
certificate        0
genre              0
director           0
cast               0
votes              0
review_count       0
dtype: int64


,title,year,certificate,duration_,genre,rating,metascore,director,cast,votes,review_count
0,The Idea of You,2023,R,115.0,"Comedy, Drama, Romance",6.4,67.0,Michael Showalter,"Anne Hathaway, Nicholas Galitzine, Ella Rubin,...","28,744",166
1,Kingdom of the Planet of the Apes,2023,PG-13,145.0,"Action, Adventure, Sci-Fi",7.3,66.0,Wes Ball,"Owen Teague, Freya Allan, Kevin Durand, Peter ...","22,248",183
2,Unfrosted,2023,PG-13,97.0,"Biography, Comedy, History",5.5,42.0,Jerry Seinfeld,"Isaac Bae, Jerry Seinfeld, Chris Rickett, Rach...","18,401",333


## Outlier handling with IQR

Cap extreme values in numeric columns (including converted ones) using the IQR rule.


In [6]:
# Work on all numeric columns
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()

def cap_outliers_iqr(series: pd.Series) -> pd.Series:
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    return series.clip(lower, upper)

for col in num_cols:
    df[col] = cap_outliers_iqr(df[col])

df.head(3)

,title,year,certificate,duration_,genre,rating,metascore,director,cast,votes,review_count
0,The Idea of You,2023.0,R,115.0,"Comedy, Drama, Romance",6.4,67.0,Michael Showalter,"Anne Hathaway, Nicholas Galitzine, Ella Rubin,...","28,744",166
1,Kingdom of the Planet of the Apes,2023.0,PG-13,145.0,"Action, Adventure, Sci-Fi",7.3,66.0,Wes Ball,"Owen Teague, Freya Allan, Kevin Durand, Peter ...","22,248",183
2,Unfrosted,2023.0,PG-13,97.0,"Biography, Comedy, History",5.5,42.0,Jerry Seinfeld,"Isaac Bae, Jerry Seinfeld, Chris Rickett, Rach...","18,401",333


## Parse and clean genre and cast

- Split comma-separated values into lists
- Strip whitespace
- Handle nulls and 'Unknown' gracefully


In [7]:
def split_clean_list(val):
    if pd.isna(val):
        return []
    s = str(val)
    if s.strip().lower() == 'unknown':
        return []
    return [x.strip() for x in s.split(',') if x.strip()]

if 'genre' in df.columns:
    df['genre'] = df['genre'].apply(split_clean_list)
if 'cast' in df.columns:
    df['cast'] = df['cast'].apply(split_clean_list)

df.head(3)

,title,year,certificate,duration_,genre,rating,metascore,director,cast,votes,review_count
0,The Idea of You,2023.0,R,115.0,"[Comedy, Drama, Romance]",6.4,67.0,Michael Showalter,"[Anne Hathaway, Nicholas Galitzine, Ella Rubin...","28,744",166
1,Kingdom of the Planet of the Apes,2023.0,PG-13,145.0,"[Action, Adventure, Sci-Fi]",7.3,66.0,Wes Ball,"[Owen Teague, Freya Allan, Kevin Durand, Peter...","22,248",183
2,Unfrosted,2023.0,PG-13,97.0,"[Biography, Comedy, History]",5.5,42.0,Jerry Seinfeld,"[Isaac Bae, Jerry Seinfeld, Chris Rickett, Rac...","18,401",333


In [8]:
# Rename duration_ to duration if present
if 'duration_' in df.columns:
    df = df.rename(columns={'duration_': 'duration'})

# Convert duration to integer
if 'duration' in df.columns:
    df['duration'] = pd.to_numeric(df['duration'], errors='coerce')
    df['duration'] = df['duration'].fillna(df['duration'].median()).astype(int)


## Create final DataFrames

- `df_clean`: fully cleaned dataset
- `df_cast`: exploded by cast for actor-level analysis
- `df_genre`: exploded by genre for type-level analysis


In [9]:
# Copy cleaned dataset
df_clean = df.copy()

# Explode cast
if 'cast' in df_clean.columns:
    df_cast = df_clean.explode('cast').reset_index(drop=True)
else:
    df_cast = pd.DataFrame(columns=df_clean.columns.tolist())

# Explode genre
if 'genre' in df_clean.columns:
    df_genre = df_clean.explode('genre').reset_index(drop=True)
else:
    df_genre = pd.DataFrame(columns=df_clean.columns.tolist())

# Print shapes for verification
print("df_clean shape:", df_clean.shape)
print("df_cast shape:", df_cast.shape)
print("df_genre shape:", df_genre.shape)

# Preview each DataFrame
print("Cleaned dataset preview:")
display(df_clean.head(3))

print("Exploded cast preview:")
display(df_cast.head(3))

print("Exploded genre preview:")
display(df_genre.head(3))


df_clean shape: (10000, 11)
df_cast shape: (40758, 11)
df_genre shape: (25128, 11)
Cleaned dataset preview:


,title,year,certificate,duration,genre,rating,metascore,director,cast,votes,review_count
0,The Idea of You,2023.0,R,115,"[Comedy, Drama, Romance]",6.4,67.0,Michael Showalter,"[Anne Hathaway, Nicholas Galitzine, Ella Rubin...","28,744",166
1,Kingdom of the Planet of the Apes,2023.0,PG-13,145,"[Action, Adventure, Sci-Fi]",7.3,66.0,Wes Ball,"[Owen Teague, Freya Allan, Kevin Durand, Peter...","22,248",183
2,Unfrosted,2023.0,PG-13,97,"[Biography, Comedy, History]",5.5,42.0,Jerry Seinfeld,"[Isaac Bae, Jerry Seinfeld, Chris Rickett, Rac...","18,401",333


Exploded cast preview:


,title,year,certificate,duration,genre,rating,metascore,director,cast,votes,review_count
0,The Idea of You,2023.0,R,115,"[Comedy, Drama, Romance]",6.4,67.0,Michael Showalter,Anne Hathaway,"28,744",166
1,The Idea of You,2023.0,R,115,"[Comedy, Drama, Romance]",6.4,67.0,Michael Showalter,Nicholas Galitzine,"28,744",166
2,The Idea of You,2023.0,R,115,"[Comedy, Drama, Romance]",6.4,67.0,Michael Showalter,Ella Rubin,"28,744",166


Exploded genre preview:


,title,year,certificate,duration,genre,rating,metascore,director,cast,votes,review_count
0,The Idea of You,2023.0,R,115,Comedy,6.4,67.0,Michael Showalter,"[Anne Hathaway, Nicholas Galitzine, Ella Rubin...","28,744",166
1,The Idea of You,2023.0,R,115,Drama,6.4,67.0,Michael Showalter,"[Anne Hathaway, Nicholas Galitzine, Ella Rubin...","28,744",166
2,The Idea of You,2023.0,R,115,Romance,6.4,67.0,Michael Showalter,"[Anne Hathaway, Nicholas Galitzine, Ella Rubin...","28,744",166


## Save cleaned outputs (optional)

Uncomment the lines below to save cleaned CSVs for downstream work.


In [10]:
df_clean.to_csv('imdb_clean.csv', index=False)
df_cast.to_csv('imdb_cast_exploded.csv', index=False)
df_genre.to_csv('imdb_genre_exploded.csv', index=False)